# 0 Configuration

In [ ]:
config = {
    "modeling": {
        "dummy": False,
        "linear_reg": False,
        "svr": False,
        "tree": False,
        "forest": False,
        "xgboost": True
    },
    "explain": {
        "setup": True,
        "bar": True,
        "force": False,
        "waterfall": False,
        "summary": False,
        "dependence": False
    },
}

***
# 1 Dependency import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import mutual_info_regression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import set_config

import xgboost
from xgboost import XGBRegressor

import shap
shap.initjs()

# plt.style.use(["dark_background"])
plt.style.use(["default"])
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

np.random.seed(0)

In [ ]:
def get_categorical_features_name(dataset, split_by_unique_count=True, split_count=10):
    features_name = dataset.select_dtypes(["object", "bool"]).columns
    if split_by_unique_count:
        less_uniques = [feature_name for feature_name in features_name if dataset[feature_name].nunique() <= split_count]
        lot_uniques = features_name.difference(less_uniques).tolist()
        return (less_uniques, lot_uniques)
    else:
        return features_name.values

In [ ]:
def get_numerical_features_name(dataset):
    features_name = dataset.select_dtypes(["int64", "float64"]).columns.values.tolist()
    return features_name

In [ ]:
class EvaluationResult():
    def __init__(self, gridsearch, target, X_train, y_train, X_test, y_test, num_cols, cat_less_unique_cols, cat_lot_unique_cols):
        self.__gs = gridsearch
        self.__X_train = X_train
        self.__y_train = y_train
        self.__X_test = X_test
        self.__y_test = y_test
        self.__target = target
        self.__score = self.__gs.score(self.__X_test, self.__y_test)
        self.__ratio = np.abs((self.__score * 100) / self.__y_test.mean())
        self.__num_cols = num_cols
        self.__cat_less_unique_cols = cat_less_unique_cols
        self.__cat_lot_unique_cols = cat_lot_unique_cols
        self.__cat_less_unique_cols_preproc = self.__gs.best_estimator_["transforms"].transformers_[1][1]["one_hot_encoder"].get_feature_names_out(self.__cat_less_unique_cols).tolist()

        self.best_model = self.__gs.best_estimator_.named_steps["model"]
        self.feature_names = self.__num_cols + self.__cat_less_unique_cols_preproc + self.__cat_lot_unique_cols

        self.X_train_transform = pd.DataFrame(data=self.__gs.best_estimator_.named_steps["transforms"].transform(self.__X_train), columns=self.feature_names)
        self.X_test_transform = pd.DataFrame(data=self.__gs.best_estimator_.named_steps["transforms"].transform(self.__X_test), columns=self.feature_names)

    def print_metrics(self):
        print(f"RMSE: {-self.__score:.4}")
        print(f"Target mean value: {self.__target.mean():.4}")
        print(f"Ratio: {self.__ratio:.4}%")
        print(f"best_params: {self.__gs.best_params_}")

In [ ]:
def evaluate(model, grid_params, dataset, target, scoring="neg_root_mean_squared_error"):
    set_config(display="diagram") # display="text" -> for textual output

    ### DATASET PREPARATION ###

    y = dataset[target].copy()
    X = dataset.drop(columns=[target]).copy()

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

    categorical_cols_less_unique, categorical_cols_lot_unique = get_categorical_features_name(X_train)
    numerical_cols = get_numerical_features_name(X_train)

    ### PIPELINE CONSTRUCTION ###

    num_pipe = Pipeline(steps=[
        ("simple_imputer", SimpleImputer(strategy="mean"))
    ])

    cat_less_unique_pipe = Pipeline(steps=[
        ("simple_imputer", SimpleImputer(strategy="most_frequent")),
        ("one_hot_encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    cat_lot_unique_pipe = Pipeline(steps=[
        ("simple_imputer", SimpleImputer(strategy="most_frequent")),
        ("ordinal_encoder", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan))
    ])

    preprocessor = ColumnTransformer([
        ("num_pipe", num_pipe, numerical_cols),
        ("cat_less_unique_pipe", cat_less_unique_pipe, categorical_cols_less_unique),
        ("cat_lot_unique_pipe", cat_lot_unique_pipe, categorical_cols_lot_unique)
    ])

    pipeline = Pipeline([
        ("transforms", preprocessor),
        ("model", model)
    ])

    ### GRIDSEARCH DECLARATION AND FITTING ###

    gs = GridSearchCV(pipeline, grid_params, scoring=scoring, refit=True)
    gs.fit(X_train, y_train)

    return EvaluationResult(gs, y, X_train, y_train, X_test, y_test, numerical_cols, categorical_cols_less_unique, categorical_cols_lot_unique)

In [ ]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

***
# 2 Loading data

In [ ]:
data = pd.read_csv("data/data-cleaned.csv", delimiter=",")

In [ ]:
data.drop(columns=["Unnamed: 0"], inplace=True)

In [ ]:
target_1 = "SiteEnergyUse(kBtu)"
target_2 = "GHGEmissionsIntensity(kgCO2e/ft2)"

In [ ]:
# y = data[target]
# X = data.drop(columns=[target])

In [ ]:
# X_train_full, X_test_full, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

In [ ]:
# categorical_cols_less_unique = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]
# categorical_cols_lot_unique = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() >= 10 and X_train_full[cname].dtype == "object"]
# numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [ ]:
# my_cols = categorical_cols_lot_unique + categorical_cols_less_unique + numerical_cols
# X_train = X_train_full[my_cols].copy()
# X_test = X_test_full[my_cols].copy()

In [ ]:
# y_test.describe()

***
# 3 Feature engineering

In [ ]:
elec_per_floor = "Electricity(kWh/fl)"
data[elec_per_floor] = (data["Electricity(kWh)"] / data["NumberofFloors"])
data[elec_per_floor].replace(np.inf, 0, inplace=True)
data[elec_per_floor].describe()

***
# 4 Modeling

## 4.1 DummyRegressor

In [ ]:
%%time
if config["modeling"]["dummy"]:

    model = DummyRegressor()

    grid_params = [
        {
            "model__strategy": ["mean", "median"]
        },
        {
            "model__strategy": ["quantile"],
            "model__quantile": np.arange(0, 1.1, 0.1),
        }
    ]

    eval_result = evaluate(model, grid_params, data, target_1)

    eval_result.print_metrics()

***
## 4.2 LinearRegression

In [ ]:
%%time
if config["modeling"]["linear_reg"]:

    model = LinearRegression()

    grid_params = [
        {
            "model__fit_intercept": [True]
        }
    ]

    eval_result = evaluate(model, grid_params, data, target_1)

    eval_result.print_metrics()

***
## 4.3 SupportVectorRegression

In [ ]:
%%time
if config["modeling"]["svr"]:

    model = SVR()

    grid_params = [
        {
            "model__kernel": ["rbf"],
            "model__degree": [3],
            "model__gamma": ["scale"]
        }
    ]

    eval_result = evaluate(model, grid_params, data, target_1)

    eval_result.print_metrics()

***
## 4.4 DecisionTreeRegressor

In [ ]:
%%time
if config["modeling"]["tree"]:

    model = DecisionTreeRegressor()

    grid_params = [
        {
            "model__random_state": [1],
            "model__max_depth": [2, 3, 4],
            "model__min_samples_leaf": range(1, 11, 1),
            "model__criterion": ["mse"]
        }
    ]

    eval_result = evaluate(model, grid_params, data, target_1)

    eval_result.print_metrics()

***
## 4.5 RandomForestRegressor

In [ ]:
%%time
if config["modeling"]["forest"]:

    model = RandomForestRegressor()

    grid_params = [
        {
            "model__random_state": [1],
            "model__n_estimators": [25],
            "model__min_samples_leaf": [1],
            "model__criterion": ["mse"]
        }
    ]

    eval_result = evaluate(model, grid_params, data, target_1)

    eval_result.print_metrics()

***
## 4.6 XGBRegressor

In [ ]:
%%time
if config["modeling"]["xgboost"]:

    model = XGBRegressor()

    grid_params = [
        {
            "model__random_state": [1],
            "model__max_depth": [6], # 6, 3 .. 10
            "model__n_estimators": [100], # 100, 100 .. 1000
            "model__learning_rate": [0.04], # 0.3, 0.01 .. 0.3
            "model__colsample_bytree": [1], # 1, 0.5 .. 1
            "model__subsample": [1], # 1, 0.6 .. 1
            "model__alpha": [0], # 0
            "model__lambda": [1], # 1
            "model__gamma": [0], # 0
        }
    ]

    eval_result = evaluate(model, grid_params, data, target_1)

    eval_result.print_metrics()

RMSE: 0.1126  
Target mean value: 0.621  
Ratio: 19.9%  
best_params: {'model__alpha': 0, 'model__colsample_bytree': 1, 'model__gamma': 0, 'model__lambda': 1, 'model__learning_rate': 0.04, 'model__max_depth': 6, 'model__n_estimators': 100, 'model__random_state': 1, 'model__subsample': 1}  
CPU times: user 2min 56s, sys: 3.86 s, total: 2min 59s  
Wall time: 16.4 s

RMSE: 0.1177  
Target mean value: 0.621  
Ratio: 20.81%  
best_params: {'model__alpha': 0, 'model__colsample_bytree': 1, 'model__gamma': 0, 'model__lambda': 1, 'model__learning_rate': 0.05, 'model__max_depth': 6, 'model__n_estimators': 100, 'model__random_state': 1, 'model__subsample': 1}  
CPU times: user 2min 20s, sys: 4.11 s, total: 2min 24s  
Wall time: 13.1 s

RMSE: 0.1331  
Target mean value: 0.621  
Ratio: 23.53%  
best_params: {'model__learning_rate': 0.1, 'model__max_depth': 6, 'model__n_estimators': 100, 'model__random_state': 1}  
CPU times: user 2min 52s, sys: 4.62 s, total: 2min 56s  
Wall time: 16.2 s

***
# 5 Feature explainability

## 5.1 Explainer setup

In [ ]:
%%time
if config["explain"]["setup"]:
    explainer = shap.TreeExplainer(eval_result.best_model, data=eval_result.X_train_transform)
    explainer_bis = explainer(eval_result.X_train_transform)
    shap_values = explainer.shap_values(eval_result.X_train_transform)

***
## 5.2 Feature importance

In [ ]:
%%time
if config["explain"]["bar"]:
    shap.plots.bar(explainer_bis)

In [ ]:
%%time
if config["explain"]["force"]:
    shap.force_plot(explainer.expected_value, shap_values[0,:], eval_result.X_train_transform.iloc[0,:], matplotlib=True)

In [ ]:
%%time
if config["explain"]["waterfall"]:
    shap.plots.waterfall(explainer_bis[0])

***
## 5.3 Summary

In [ ]:
%%time
if config["explain"]["summary"]:
    shap.summary_plot(shap_values, eval_result.X_train_transform)

***
## 5.4 Dependence

In [ ]:
%%time
if config["explain"]["dependence"]:
    shap.dependence_plot("Electricity(kWh)", shap_values, eval_result.X_train_transform)

In [ ]:
%%time
if config["explain"]["dependence"]:
    shap.dependence_plot("NumberofFloors", shap_values, eval_result.X_train_transform)

In [ ]:
%%time
if config["explain"]["dependence"]:
    shap.dependence_plot("SiteEUI(kBtu/sf)", shap_values, eval_result.X_train_transform)

In [ ]:
%%time
if config["explain"]["dependence"]:
    shap.dependence_plot("Latitude", shap_values, eval_result.X_train_transform)

In [ ]:
%%time
if config["explain"]["dependence"]:
    shap.dependence_plot("Longitude", shap_values, eval_result.X_train_transform)

In [ ]:
%%time
if config["explain"]["dependence"]:
    shap.dependence_plot("CouncilDistrictCode", shap_values, eval_result.X_train_transform)

In [ ]:
%%time
if config["explain"]["dependence"]:
    shap.dependence_plot("YearBuilt", shap_values, eval_result.X_train_transform)

***
# 6 ...